In [1]:
import sys
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2

base_path=Path("d:/")
base_path = base_path / "work" / "projects"
for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import airsim
from ezai_util import DictObj

D:\work\projects\ezai_util
D:\work\projects\ai_coop_py


In [2]:
uenv_path = Path("d:/") / 'work' / 'unity-envs'
uenv_file = uenv_path / "airsim_win_unlimited" / 'AirSimAssets'
uenv_file = str(uenv_file.resolve())
#uenv_file = None

In [3]:
rl_conf = DictObj({
  "env_name":'airsim',
  "seed": 123,
  "discount": 0.99,
  "tau": 5e-3,
  "expl_noise": 0.1,
  "batch_size": 256,
  "batches_before_train": 2,
  "memory_capacity":10000,
  "episode_max_steps":100,
  "num_episodes":2
})

env_conf = DictObj({
    'observation_mode' : 0,
    'max_steps' : rl_conf.episode_max_steps,
    'reward_for_goal' : 50,
    'reward_for_ep': .005,
    'reward_for_other' : -.1,
    'reward_for_falling_off_map' :  -50,
    'reward_for_step': -.0001,
    'segmentation_mode': 1
})

In [5]:
env = airsim.AirSimEnv(filename=uenv_file, observation_mode = 0, max_steps = rl_conf.episode_max_steps)
env.info()

2020-10-16 14:28:36 INFO [environment.py:107] Connected to Unity environment with package version 1.1.0-preview and communication version 1.0.0
2020-10-16 14:28:37 INFO [environment.py:265] Connected new brain:
VisualNavigator?team=0


Env Info
-----------
Action Space: Box(2,)
Action sample: [ 0.39293838 -0.42772132]
Action Space Shape: (2,)
Action Space Low: [-1. -1.]
Action Space High: [1. 1.]
Observation Mode: 0
Gym Observation Space: Tuple(Box(10,))
Gym Observation Space Shape: None
Self Observation Space: Tuple(Box(10,))
Self Observation Space Shape: None
Reward Range: (-inf, inf)
Metadata: {'render.modes': ['rgb_array']}
Initial State: [array([ 2.4926619e+03,  4.0132114e+01, -2.9184119e+02,  1.4640316e-03,
        2.1606718e-01, -4.1355323e-03, -9.7636873e-01,  2.6203770e+03,
        4.6497551e+01, -3.2535733e+02], dtype=float32)]
First Step State: ([array([ 2.4926619e+03,  4.0136219e+01, -2.9184247e+02,  4.1413079e-03,
        2.1582977e-01, -8.8777961e-03, -9.7638184e-01,  2.6203770e+03,
        4.6497551e+01, -3.2535733e+02], dtype=float32)], -1e-04, False, {'step': <mlagents_envs.base_env.DecisionSteps object at 0x0000018007E03D60>})


c:\Miniconda3\envs\ezai\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
trainer = airsim.Trainer(rl_conf,env,enable_logging=False)

2020-10-16 14:28:45 WARNING [__init__.py:270] Could not seed environment VisualNavigator?team=0


In [7]:
trainer.train()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.59it/s]


In [8]:
trainer.agent.save_actor('model.onnx')

In [9]:
env.close()

2020-10-16 14:28:51 INFO [environment.py:417] Environment shut down with return code 0 (CTRL_C_EVENT).
